In [1]:
import cv2 as cv
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

import ipywidgets as widgets
from typing import Optional

import detector.detect as detect

%matplotlib inline

images_root = Path.cwd()/"test-images"
frames_root = images_root/"video-frames"

In [2]:
video = cv.VideoCapture((images_root/"ring-video.mp4").as_posix())

frames = []
while video.isOpened():
    ok, frame = video.read()
    if not ok:
        break
    frame = cv.resize(frame, None, fx = 0.5, fy=0.5)
    frame = cv.cvtColor(frame, cv.COLOR_BGR2HLS_FULL)
    frames.append(frame)

video.release()

In [3]:
def change_format(image, conversion):
    return cv.cvtColor(image, conversion)

def show_filtered(frame_id, hue, sat, light):
    frame = frames[frame_id]
    lower = np.array([hue[0], light[0], sat[0]])
    upper = np.array([hue[1], light[1], sat[1]])
    image = detect.filter_image(frame, lower, upper)
    plt.imshow(change_format(image, cv.COLOR_HLS2RGB_FULL))

def show_original(frame_id):
    frame = frames[frame_id]
    image = change_format(frame, cv.COLOR_HLS2RGB_FULL)
    plt.imshow(image)

def show_match(frame_id, hue, sat, light):
    frame = frames[frame_id]
    lower = np.array([hue[0], light[0], sat[0]])
    upper = np.array([hue[1], light[1], sat[1]])
    matches = detect.ring(frame, lower, upper)
    output = change_format(frame, cv.COLOR_HLS2RGB_FULL)
    for m in matches:
        output = m.show(output, line_weight=2)
    plt.imshow(output)

In [4]:
frame_selector = widgets.IntSlider(description="Frame", min=0, max=len(frames)-1, step=1)

hue = widgets.IntRangeSlider(description="Hue", min=0, max=255, value=[7,32])
sat = widgets.IntRangeSlider(description="Saturation", min=0, max=255, value=[149,255])
light = widgets.IntRangeSlider(description="Lightness", min=0, max=255, value=[85,216])
sliders = widgets.VBox([frame_selector, hue, light, sat])
prev = widgets.Button(description="prev")
def prev_frame(_button):
    frame_selector.value = frame_selector.value - 1
prev.on_click(prev_frame)
next = widgets.Button(description="next")
def next_frame(_button):
    frame_selector.value = frame_selector.value + 1
next.on_click(next_frame)

filtered = widgets.interactive_output(show_filtered, {
    "frame_id": frame_selector,
    "hue": hue,
    "light": light,
    "sat": sat,
})
original = widgets.interactive_output(show_original, {
    "frame_id": frame_selector,
})
matched = widgets.interactive_output(show_match, {
    "frame_id": frame_selector,
    "hue": hue, #firm [7,32]
    "light": light, #[85,216]
    "sat": sat, #[149,255]
})
widgets.VBox([widgets.HBox([sliders, prev, next]), widgets.HBox([original, filtered]), matched])